In [ ]:
# Load libraries
import numpy as np
from pickle import dump
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dense, Conv1D, Flatten, AveragePooling1D
import pandas as pd
from pandas import read_csv
from sklearn.model_selection import train_test_split


from google.colab import drive
#drive.mount('/content/drive')
drive.mount("/content/drive", force_remount = True)


num_rounds= 3
url =  "/content/drive/MyDrive/datafile/Dataset_salsa_100000_3round.csv"


names=['del_0','del_1','del_2','del_3', 'del_4', 'del_5','del_6','del_7','del_8','del_9','del_10', 'del_11', 'del_12','del_13','del_14','del_15',
       'C1_0','C1_1','C1_2', 'C1_3','C1_4','C1_5','C1_6', 'C1_7','C1_8','C1_9','C1_10', 'C1_11','C1_12','C1_13','C1_14', 'C1_15',
       'C2_0','C2_1','C2_2', 'C2_3','C2_4','C2_5','C2_6', 'C2_7','C2_8','C2_9','C2_10', 'C2_11','C2_12','C2_13','C2_14', 'C2_15', 'Label']
dataset=pd.read_csv(url,names=names,engine='python',encoding='ISO-8859-1')


#dataset1=dataset.drop(['PT'],axis=1)
dataset1=dataset.iloc[1:]


print(len(dataset1.iloc[0][0]))



In [ ]:




D = {i: dataset1[f'del_{i}'].to_numpy(dtype=np.uint32) for i in range(16)}
C1 = {i: dataset1[f'C1_{i}'].to_numpy(dtype=np.uint32) for i in range(16)}
C2 = {i: dataset1[f'C2_{i}'].to_numpy(dtype=np.uint32) for i in range(16)}

C = [D[i] for i in range(16)] + [C1[i] for i in range(16)] + [C2[i] for i in range(16)]




Y=dataset1['Label'].to_numpy()
Y=Y.astype(np.int32)

In [ ]:
def WORD_SIZE():
  return(32);
def convert_to_binary(arr):
  X1 = np.zeros((48 * WORD_SIZE(),len(arr[0])),dtype=np.uint8);
  for i in range(48 * WORD_SIZE()):
    index = i // WORD_SIZE();
    offset = WORD_SIZE() - (i % WORD_SIZE()) - 1;
    X1[i] = ((arr[index]) >> offset) & 1;

  #print(X1)
  X1 = X1.transpose();

  return(X1);

X=convert_to_binary(C)
print(len(X[0]))

In [ ]:
#Split the data into train test set#
import numpy as np
seed=9
np.random.seed(seed)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.50,random_state=seed)
#X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.50,random_state=seed)

print(len(X_train),len(X_test))

y_test

In [ ]:
#Create the CNN Model#
depth=3      # Depth of the CNN network

#from keras.layers.normalization.batch_normalization import BatchNormalization
from keras.layers import BatchNormalization, Conv1D, AveragePooling1D, Flatten, Dense
from keras.regularizers import l2

reg_param=0.0001
model=Sequential()
#Add initial
model.add(Conv1D(filters=32, kernel_size=1, padding='same', kernel_regularizer=l2(reg_param),activation='relu', input_shape=(1536,1)))  # Change the input shape accordingly
model.add(BatchNormalization())


for i in range(depth):
  model.add(Conv1D(filters=32, kernel_size=3, padding='same', kernel_regularizer=l2(reg_param), activation='relu'))
  model.add(BatchNormalization())

  model.add(AveragePooling1D(pool_size = 3))

  model.add(Conv1D(filters=32, kernel_size=3, padding='same', kernel_regularizer=l2(reg_param), activation='relu'))
  model.add(BatchNormalization())

  model.add(AveragePooling1D(pool_size = 3))

  model.add(Conv1D(filters=32, kernel_size=3, padding='same', kernel_regularizer=l2(reg_param), activation='relu'))
  model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(64, activation='relu',kernel_regularizer=l2(reg_param),kernel_initializer='glorot_uniform'))
model.add(BatchNormalization())

model.add(Dense(64, activation='relu',kernel_regularizer=l2(reg_param),kernel_initializer='glorot_uniform'))
model.add(BatchNormalization())

model.add(Dense(1, kernel_regularizer=l2(reg_param),activation='sigmoid'))


In [ ]:
print(model.summary())

In [ ]:
# compile the keras model
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [ ]:
print(len(X_test))
c0=1
c1=0
for i in y_test:
  if(i==1):
    c1=c1+1
  else:
        c0=c0+1
print(c0,c1)

In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping
early_stopping_monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True
)
def make_checkpoint(datei):
  res = ModelCheckpoint(datei, monitor='val_loss', save_best_only = True);
  return(res);
#set up model checkpoint
wdir = './freshly_trained_nets/'
check = make_checkpoint(wdir+'best'+str(num_rounds)+'depth'+str(depth)+'.keras');



# Learning rate scheduler
from keras.callbacks import LearningRateScheduler
def cyclic_lr(num_epochs, high_lr, low_lr):
  res = lambda i: low_lr + ((num_epochs-1) - i % num_epochs)/(num_epochs-1) * (high_lr - low_lr);
  return(res);

lr = LearningRateScheduler(cyclic_lr(10,0.002, 0.0001));

#callbacks = [LearningRateScheduler(lr, verbose=1)]


from time import time
start = time()


h = model.fit(X_train,y_train,callbacks=[lr,check],epochs= 10 ,batch_size= 50,validation_data=(X_test, y_test));
print(time()-start)

In [ ]:
# make probability predictions with the model
predictions = model.predict(X_test)
# round predictions
#rounded = [(x[0]) for x in predictions]


prd=[]
for i in range(len(predictions)):
  if predictions[i]>0.5:
    prd.append(1)
    #print(predictions)
  else:
    prd.append(0)


from sklearn import metrics
matrix = metrics.confusion_matrix(y_test[:len(prd)], prd)
print(matrix)

In [ ]:
TP=matrix[0][0]
FN=matrix[0][1]
FP=matrix[1][0]
TN=matrix[1][1]
TPR=TP/(TP+FN)
print(TPR)
TNR=TN/(TN+FP)
print(TNR)

In [ ]:
################ Data filteration for the real cipher ######################


import pandas as pd

# Load your dataset
df = pd.read_csv(url)

# Filter rows where Label is equal to 1
filtered_data = df[df['Label'] == 1]

# Save the filtered data to a new CSV file
filtered_data.to_csv("filtered_xyz.csv", index=False)



filtered_data = "filtered_xyz.csv"


c_names=['del_0','del_1','del_2','del_3', 'del_4', 'del_5','del_6','del_7','del_8','del_9','del_10', 'del_11', 'del_12','del_13','del_14','del_15',
       'C1_0','C1_1','C1_2', 'C1_3','C1_4','C1_5','C1_6', 'C1_7','C1_8','C1_9','C1_10', 'C1_11','C1_12','C1_13','C1_14', 'C1_15',
       'C2_0','C2_1','C2_2', 'C2_3','C2_4','C2_5','C2_6', 'C2_7','C2_8','C2_9','C2_10', 'C2_11','C2_12','C2_13','C2_14', 'C2_15', 'Label']
data_set=pd.read_csv(filtered_data , names = c_names,engine='python',encoding='ISO-8859-1')


data_set1= data_set.iloc[1:]

In [ ]:



D = {i: data_set1[f'del_{i}'].to_numpy(dtype=np.uint32) for i in range(16)}
C1 = {i: data_set1[f'C1_{i}'].to_numpy(dtype=np.uint32) for i in range(16)}
C2 = {i: data_set1[f'C2_{i}'].to_numpy(dtype=np.uint32) for i in range(16)}

CL = [D[i] for i in range(16)] + [C1[i] for i in range(16)] + [C2[i] for i in range(16)]

Y1 = data_set1['Label'].to_numpy()
Y1 = Y1.astype(np.int32)


X1 = convert_to_binary(CL)

column_names=['del_0','del_1','del_2','del_3', 'del_4', 'del_5','del_6','del_7','del_8','del_9','del_10', 'del_11', 'del_12','del_13','del_14','del_15',
       'C1_0','C1_1','C1_2', 'C1_3','C1_4','C1_5','C1_6', 'C1_7','C1_8','C1_9','C1_10', 'C1_11','C1_12','C1_13','C1_14', 'C1_15',
       'C2_0','C2_1','C2_2', 'C2_3','C2_4','C2_5','C2_6', 'C2_7','C2_8','C2_9','C2_10', 'C2_11','C2_12','C2_13','C2_14', 'C2_15']


print(len(X1[0]))
Y1

In [ ]:
print(len(X1))

In [ ]:
for i in range(5):  # Print first 5 rows across arrays
    print([arr[i] for arr in CL])

In [ ]:
##################### Collecting the accurate data ##########################


import pandas as pd

# Function to convert binary array to integer
def binary_array_to_int(binary_array):
    # Convert binary array to string and then to integer
    binary_string = ''.join(str(bit) for bit in binary_array)
    return int(binary_string, 2)

In [ ]:


# Reshape features to match the input shape of your model
X1_reshaped = X1.reshape(-1, 1536, 1)



In [ ]:
print((X1_reshaped[2]))

In [ ]:
predicted=model.predict(X1_reshaped)

In [ ]:
print(len(predicted))

In [ ]:
#print(len(X_test))
c0=1
c1=0
# Create an empty list to store the indices of rows that meet the condition
filtered_indices = []
for i in range(len(predicted)):
  p=predicted[i]
  if(p>=0.5):
    filtered_indices.append(i)

In [ ]:
print(filtered_indices[0:65])
print(len(filtered_indices))

In [ ]:
# Filter X1 and Y1 based on the filtered indices
filtered_X1 = X1[filtered_indices]
filtered_Y1 = Y1[filtered_indices]

# Convert each row of binary values to integers and store in a list
int_values = [[binary_array_to_int(row[i:i+32]) for i in range(0, 1536, 32)] for row in filtered_X1 ]

# Assuming you have the column names stored in a list 'column_names'
# If not, you can replace it with your actual column names
#column_names = ['Feature1', 'Feature2', ...]  # Replace with your column names

# If you want to store the filtered X and Y to new CSV files:
# First, convert filtered_X and filtered_Y back to a DataFrame
filtered_df = pd.DataFrame(int_values, columns= column_names)
filtered_df['Label'] = filtered_Y1  # Add the labels to the DataFrame if needed

# Save the header to a separate variable
header = filtered_df.columns.tolist()

# Save the filtered data to a new CSV file
filtered_df.to_csv("filtered_data.csv", index=False, header=header)


In [ ]:
#####################################################   End  ##############################################################3